In [ ]:
import numpy as np
import os
from skimage import io
import pandas as pd
from COLD import * 
from LBP import * 
from HOG import *

In [ ]:
malesDirectory = 'preprocessed/Males'
femalesDirectory = 'preprocessed/Females'

featuresTable = []
for imgPath in os.listdir(malesDirectory):
    img = io.imread(malesDirectory+'/'+imgPath)
    cold_r,cold_theta,cold_distances,cold_pca_slope = ColdFeature(img)
    lbpHist = extract_lbp(img).flatten().tolist()
    hogfeature = calculateHOG(img)
    if not math.isnan(cold_distances):
        dataValues = [1,hogfeature, cold_r,cold_theta,cold_distances,cold_pca_slope]
        dataValues.extend(lbpHist)
        featuresTable.append(dataValues)


for imgPath in os.listdir(femalesDirectory):
    img = io.imread(femalesDirectory+'/'+imgPath)
    cold_r,cold_theta,cold_distances,cold_pca_slope = ColdFeature(img)
    lbpHist = extract_lbp(img).flatten().tolist()
    hogfeature = calculateHOG(img)
    if not math.isnan(cold_distances):
        dataValues = [0,hogfeature, cold_r,cold_theta,cold_distances,cold_pca_slope]
        dataValues.extend(lbpHist)
        featuresTable.append(dataValues)

LBPLabels = ['LBP_HIST_'+str(i) for i in range(25)]
labels = ['Gender', 'HOG', 'COLD_AVG_R','COLD_AVG_Theta' ,'COLD_AVG_Distances','COLD_PCA_SLOPE']
labels.extend(LBPLabels)
featuresTable = np.array(featuresTable)
df = pd.DataFrame(data=featuresTable,columns=labels)

#Standard Normalization

#Remove columns where std = 0 (for lbp)
df = df.loc[:, (df != 0).any(axis=0)]

#Calculate standard normalization
for column in df:
    if column != 'Gender':
        df[column] = (df[column] -
                           df[column].mean()) / df[column].std()
df.to_csv('dataset.csv')